In [1]:
import os
import sys
sys.path.append('../')
sys.path.append('../datasets')
sys.path.append('../models')
import torch
import torchvision
import torch.backends.cudnn as cudnn
import random
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from importlib import reload
from datasets.three_dim_shapes import ThreeDimShapesDataset
from datasets.small_norb import SmallNORBDataset
from datasets.seq_mnist import SequentialMNIST
import models.seqae as seqae
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from datasets import so3_data as so3d
from einops import rearrange
from sklearn.metrics import r2_score
import pdb
from models.base_networks import MLP_iResNet, LinearNet, MLP
from utils import optimize_bd_cob as obc
from importlib import reload
import copy
from torch import nn

import csv
import ast
from source import yaml_utils as yu


from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
else:
    device = torch.device('cpu')
    gpu_index = -1

    
    
from utils import yaml_utils as yu
import yaml
result_dir = '/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result'

rootpath = '/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/'
datpath = '/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/datasets/so3'







In [2]:
unko= nn.Linear(3, 3)
nn.init.xavier_uniform(unko.weight.data)

/tmp/ipykernel_178976/2125234506.py:2: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(unko.weight.data)


tensor([[ 0.2994,  0.9216, -0.4582],
        [-0.2367, -0.0527,  0.6424],
        [-0.9349, -0.4029,  0.3092]])

In [3]:
weightmat  = unko.weight.data

### If  SvD  is to agree with eigen, this is required
#weightmat = ( weightmat + weightmat.permute([1,0]))/2

In [4]:
torch.linalg.cond(weightmat)

tensor(2.9367)

In [21]:
aaa = torch.linalg.svd(weightmat)
aaa

torch.return_types.linalg_svd(
U=tensor([[-0.6659,  0.7411,  0.0859],
        [ 0.3387,  0.1978,  0.9199],
        [ 0.6647,  0.6416, -0.3827]]),
S=tensor([1.4666, 0.5933, 0.4994]),
Vh=tensor([[-0.6143, -0.6132,  0.4966],
        [-0.7159,  0.6978, -0.0239],
        [ 0.3319,  0.3701,  0.8677]]))

In [22]:

torch.real(torch.linalg.eig(weightmat).eigenvalues)

tensor([-0.5738,  0.5649,  0.5649])

In [23]:
# def initialize_linear(mylayer, thresh = 1.5):

#     nn.init.xavier_uniform_(mylayer.weight.data)
#     weightmat  = mylayer.weight.data

#     while torch.linalg.cond(weightmat) > thresh:
#         nn.init.xavier_uniform_(mylayer.weight.data)
#         weightmat = mylayer.weight.data
#         weightmat  = weightmat / torch.sqrt(torch.sum(weightmat @weightmat.permute([1,0])))

#     mylayer.weight.data = nn.Parameter(weightmat)

In [24]:
# random_vec_r = torch.tensor(np.random.uniform(size=(3,1))).float() 
# random_vec_r = random_vec_r / torch.sqrt(torch.sum(random_vec_r**2))


# repeats = 10000
# for k in range(repeats):
    
#     random_vec_l = weightmat.permute([1,0]) @ random_vec_r 
#     random_vec_l = random_vec_l / torch.sqrt(torch.sum(random_vec_l**2))
#     random_vec_l = random_vec_l.permute([1,0])
    
#     random_vec_r = random_vec_l @ weightmat
#     random_vec_r = random_vec_r / torch.sqrt(torch.sum(random_vec_r**2))
#     random_vec_r = random_vec_r.permute([1,0])


# print((random_vec_l @ weightmat @ random_vec_r))

In [25]:
( aaa.U @torch.diag(aaa.S) )/ aaa.U

tensor([[1.4666, 0.5933, 0.4994],
        [1.4666, 0.5933, 0.4994],
        [1.4666, 0.5933, 0.4994]])

In [37]:
random_vec_r = torch.tensor(np.random.uniform(size=(3,1))).float() 
random_vec_r = random_vec_r / torch.sqrt(torch.sum(random_vec_r**2))

random_vec_l = torch.tensor(np.random.uniform(size=(1,3))).float() 
random_vec_l = random_vec_l / torch.sqrt(torch.sum(random_vec_l**2))


repeats = 100
for k in range(repeats):
    
    random_vec_r = weightmat.permute([1,0]) @ random_vec_r 
    random_vec_r = random_vec_r / torch.sqrt(torch.sum(random_vec_r**2))
    
    random_vec_l = random_vec_l @ weightmat
    random_vec_l = random_vec_l / torch.sqrt(torch.sum(random_vec_l**2))


print((random_vec_l @ weightmat @ random_vec_r))

tensor([[0.5864]])


In [34]:
torch.real(torch.linalg.eig(weightmat).eigenvalues)

tensor([-0.5738,  0.5649,  0.5649])

In [30]:
def spectral_norm(weightmat):
    random_vec_r = torch.tensor(np.random.uniform(size=(3,1))).float() 
    random_vec_r = random_vec_r / torch.sqrt(torch.sum(random_vec_r**2))

    random_vec_l = torch.tensor(np.random.uniform(size=(1,3))).float() 
    random_vec_l = random_vec_l / torch.sqrt(torch.sum(random_vec_l**2))


    repeats = 3
    for k in range(repeats):

        random_vec_r = weightmat.permute([1,0]) @ random_vec_r 
        random_vec_r = random_vec_r / torch.sqrt(torch.sum(random_vec_r**2))

        random_vec_l = random_vec_l @ weightmat
        random_vec_l = random_vec_l / torch.sqrt(torch.sum(random_vec_l**2))

    snorm = (random_vec_l @ weightmat @ random_vec_r)

    return snorm


snorm = spectral_norm(unko.weight.data)

#unko.weight.data  = nn.Parameter(0.97 * unko.weight.data/snorm)

In [31]:
torch.real(torch.linalg.eig(unko.weight.data).eigenvalues)

tensor([-0.5738,  0.5649,  0.5649])

In [32]:
snorm

tensor([[0.7622]])